##Import packages and read in data from .csv

In [ ]:
#@title  { vertical-output: false, display-mode: "form" }
#import relevant packages
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from google.colab import widgets as g_widgets
from google.colab import output
from matplotlib import pylab
from mpl_toolkits.mplot3d import Axes3D
!pip install --upgrade ipywidgets==8.0.4
import ipywidgets as widgets
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
!pip install geopandas
import geopandas as gpd
import shapely.geometry
!pip install --upgrade plotly==5.5.0
import plotly.express as px
import plotly.graph_objs as go

#load in data from Git repo
url = 'https://raw.githubusercontent.com/difuse-dartmouth/21s_ENVS3/main/Module_Data/parish_data_v7.csv'
df = pd.read_csv(url)

#remove irrelevant fields for analysis
df2 = df.drop(columns=['FIPS', 'Parish Name'])
feature_names = list(df2.columns)

#define dataframes for each covid wave where other waves are excluded
First_Covid_Wave = df2.drop(columns=['Second Wave of Covid Deaths per 10k',
                                     'Third Wave of Covid Deaths per 10k',
                                     'Fourth Wave of Covid Deaths per 10k',
                                     'Covid Deaths per 10k thru 9/26/21'])
Second_Covid_Wave = df2.drop(columns=['First Wave of Covid Deaths per 10k',
                                      'Third Wave of Covid Deaths per 10k',
                                      'Fourth Wave of Covid Deaths per 10k',
                                     'Covid Deaths per 10k thru 9/26/21'])
Third_Covid_Wave = df2.drop(columns=['First Wave of Covid Deaths per 10k',
                                     'Second Wave of Covid Deaths per 10k',
                                     'Fourth Wave of Covid Deaths per 10k',
                                     'Covid Deaths per 10k thru 9/26/21'])
Fourth_Covid_Wave = df2.drop(columns=['First Wave of Covid Deaths per 10k',
                                     'Second Wave of Covid Deaths per 10k',
                                     'Third Wave of Covid Deaths per 10k',
                                     'Covid Deaths per 10k thru 9/26/21'])
Total_Covid_Deaths_Sept_26 = df2.drop(columns=['First Wave of Covid Deaths per 10k',
                                               'Second Wave of Covid Deaths per 10k',
                                     'Third Wave of Covid Deaths per 10k',
                                     'Fourth Wave of Covid Deaths per 10k'])

#read in spatial data for map from Git repo into geopandas dataframe
url2 = 'https://github.com/difuse-dartmouth/21s_ENVS3/blob/main/Module_Data/git_shapefiles/parish_data_v8.json?raw=true'
parish_data = gpd.read_file(url2)
parish_data.sort_values(by=['FIPS'], inplace=True)

#define centroids
parish_data["center"] = parish_data.geometry.centroid
LA_parishes = parish_data[['parish_nam','geometry']].set_index("parish_nam")
map_variables = list(parish_data.columns)

#remove irrelevant fields for analysis
removed_cols = ['FID','FIPS','parish_nam', 'geometry','center', 'FIPS_1']
for item in removed_cols:
  map_variables.remove(item)

# Scale data to use for bubble layer
data = parish_data[[m for m in map_variables]] #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(2,50))
data_scaled = min_max_scaler.fit_transform(data)
scaled = pd.DataFrame(data_scaled, columns=feature_names)

# Create data to use for cancer alley borders
cancer_alley_parishes = parish_data[parish_data["Cancer_All"] == 1]
cancer_alley_lats = []
cancer_alley_lons = []

for feature in cancer_alley_parishes.geometry.boundary:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        cancer_alley_lats = np.append(cancer_alley_lats, y)
        cancer_alley_lons = np.append(cancer_alley_lons, x)
        cancer_alley_lats = np.append(cancer_alley_lats, None)
        cancer_alley_lons = np.append(cancer_alley_lons, None)

#define list of variable names for covid wave dropdown in map tab
variable_list2 = [ 'First Wave of Covid Deaths per 10k',
 'Second Wave of Covid Deaths per 10k',
 'Third Wave of Covid Deaths per 10k',
 'Fourth Wave of Covid Deaths per 10k',
 'Covid Deaths per 10k thru 9/26/21']
variable_list1 = [var for var in feature_names if var not in variable_list2]

#define list of variable names for colorscales
color_scales = ["BlueYellow", "PurpleOrange", "Purples", "Reds", "RedBlue", "BlueRed"]

#define list of variable names for covid wave dropdown in correlation matrix tab
wave_list = ['First_Covid_Wave','Second_Covid_Wave','Third_Covid_Wave','Fourth_Covid_Wave',
             'Total_Covid_Deaths_Sept_26']

#define options for turning regression line off and on in regression tab
linear_regr = ['Off','On']

for var in variable_list2:
  df[var+" label"] = df["Parish Name"]+": "+df[var].round(2).astype("string")+" per 10k"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached geopandas-0.13.0-py3-none-any.whl (1.1 MB)
  Using cached Fiona-1.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 33.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


<ipython-input-1-6ad29b5445df>:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parish_data["center"] = parish_data.geometry.centroid


##Interactive Panel for Data Visualization, Plotting, and Statistical Analysis

In [ ]:
#@title  { vertical-output: false, display-mode: "form" }

#function for creating dropdowns using widget functionality in Colab
def create_dropdown(choices, first, des):
  return widgets.Dropdown(
      options = choices,
      value = choices[first],
      description = des+":",
      disabled=False,
  )

#create dropdowns for all options in each tab
widget1 = create_dropdown(feature_names, 0, "X Variable")
widget2 = create_dropdown(feature_names, 1, "Y Variable")
widget3 = create_dropdown(wave_list, 0, "Covid Wave")
widget4 = create_dropdown(['On','Off'], 1, "R Values")
widget5 = create_dropdown(['RedBlue','GreenPurple','BlueOrange'], 0, "Colorbar")
widget6 = create_dropdown(variable_list2, 0, "Covid Wave")
widget7 = create_dropdown(variable_list1, 0, "Variable")
widget8 = create_dropdown(color_scales, 2, "Colorbar")
widget9 = create_dropdown(linear_regr,0,"Regression")

#function for creating scatterplot with linear regression analysis based on selected
#x and y variables
def plot_data_linear(drop1,drop2,drop9):
#error message if user chooses same variable for x and y
  if (drop1 == drop2):
    print("X Variable cannot equal Y Variable")
    return
  f, ax = plt.subplots(figsize= [12,10])
  plt.xlabel(drop1,fontsize=20)
  plt.ylabel(drop2,fontsize=20)
  df3 = df2[[drop1,drop2]]
  df3 = df3.dropna(axis='rows')
  x = df3[[drop1]].values
  y = df3[[drop2]].values
  plt.xlabel(drop1)
  plt.ylabel(drop2)
  regr = LinearRegression()
  regr.fit(x,y)
  y_pred = regr.predict(x)
  y_inter = regr.intercept_.item()
  if np.sign(y_inter) == -1:
    sign = '-'
  else:
    sign = '+'
  slope = regr.coef_.item()
  r2 = r2_score(y, y_pred)
  std_scaler = StandardScaler()
  df_std = pd.DataFrame(std_scaler.fit_transform(df3), columns=df3.columns)
  X1 = df3[[drop1]].values
  Y1 = df3[[drop2]].values
  x1 = X1.flatten()
  y1 = Y1.flatten()
  data = pd.DataFrame({'x': x1, 'y': y1})
  model = ols("y ~ x", data).fit()
  p_value = model.pvalues[1]
  ax.plot(x,y, 'o', c='black')
  if drop9 == 'On':
    ax.plot(x,y_pred, 'r-', label="OLS")
    ax.annotate('R\N{SUPERSCRIPT TWO}= %.2f' % r2,xy=(1.01, .92),xycoords='axes fraction',
                 ha='left',fontsize ='15')
    ax.annotate('y = %.2f' % slope + 'x ' + sign + ' ' '%.2f' % y_inter,xy=(1.01, .96),xycoords='axes fraction',
                 ha='left',fontsize ='15')
    if p_value > 0.01:
      p_value2 = ('P > |t| = %.4f' % p_value)
    else:
      p_value2 = ('P > |t| = <0.01')
    ax.annotate(p_value2,xy=(1.01, .88),xycoords='axes fraction',ha='left',fontsize ='15')
    plt.grid(True)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)
    plt.show()
  else:
    plt.grid(True)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)
    plt.show()

#function for creating correlation matrix based on chosen covid wave and colorscale
#https://seaborn.pydata.org/examples/many_pairwise_correlations.html
def make_corr_plot(drop3,drop4,drop5):
  sns.set_theme(style="white")
  if drop3 == 'First_Covid_Wave':
    drop3 = First_Covid_Wave
  elif drop3 == 'Second_Covid_Wave':
    drop3 = Second_Covid_Wave
  elif drop3 == 'Third_Covid_Wave':
    drop3 = Third_Covid_Wave
  elif drop3 == 'Fourth_Covid_Wave':
    drop3 = Fourth_Covid_Wave
  else:
    drop3 = Total_Covid_Deaths_Sept_26
  std_scaler = StandardScaler()
  df_std = pd.DataFrame(std_scaler.fit_transform(drop3), columns=drop3.columns)
  corr = df_std.corr()
  f, ax = plt.subplots(figsize=(11, 9))
  ax.set_title("Correlation Matrix for Variables", fontsize = 18)
  if drop5 == 'RedBlue':
    cmap = sns.color_palette("vlag", as_cmap=True)
  elif drop5 == 'GreenPurple':
    cmap = sns.diverging_palette(145, 300, s=60, as_cmap=True)
  else:
    cmap = sns.color_palette("icefire", as_cmap=True)
    # cmap = sns.color_palette("viridis", as_cmap=True)
  if drop4 == 'On':
    sns.heatmap(corr, vmin=-1, vmax=1,cmap=cmap,center=0,annot=True, fmt=".2f",
                square=True, linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})
  else:
    sns.heatmap(corr, vmin=-1, vmax=1,cmap=cmap,center=0,square=True,
                linewidths=.5, cbar_kws={'label': 'Correlation Coefficient'})

#function for creating interactive map that displays covid waves as scaled bubbles and
#chloropleths from chosen variable and colorscale
def draw_map(drop6, drop7, drop8):
  fig = go.Figure()
  # Add choropleth layer
  color_dict = {'BlueYellow':'Viridis', 'PurpleOrange':'Inferno',
                'Purples':'Purples', 'Reds':'Reds', 'RedBlue':'Rdbu',
                'BlueRed':'IceFire'}
  fig.add_trace(go.Choroplethmapbox(geojson=LA_parishes.__geo_interface__,
                                    locations=parish_data.parish_nam,
                                    z = df[drop7],
                                    colorbar = {'title': drop7},
                                    colorscale = color_dict.get(drop8),
                                    name = drop7
                         ))
  # Add cancer alley borders
  fig.add_trace(go.Scattermapbox(mode = "lines", lat=cancer_alley_lats,
                                 lon=cancer_alley_lons, hoverinfo='skip',
                                    name = "Cancer Alley",marker=go.scattermapbox.Marker(color='DarkRed')))
  # Add bubble layer
  fig.add_trace(go.Scattermapbox(lat = parish_data.center.y,
                                 lon = parish_data.center.x, mode = 'markers',
                         marker=go.scattermapbox.Marker(size=scaled[drop6],color='Tomato'),
                         text=df[drop6+" label"],
                         hoverinfo = "text",
                         name = drop6
                         ))

  fig.update_layout(mapbox_style="carto-positron",
                  legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),
                  showlegend=True, mapbox_zoom=6,
                  mapbox_center = {"lat": 30.9843, "lon": -91.9623},
                  width=1000, height=750, margin={"r":0,"t":0,"l":0,"b":0}
                  )
  fig.layout.coloraxis.colorbar.title = str(drop6)
  fig.show()

#function for creating the different tabs and creating content in each tab
def create_tab(location):
  tb = g_widgets.TabBar(['Interactive Map','Correlation Matrix',
                         'Linear Regression',], location=location)
  with tb.output_to('Interactive Map'):
    pylab.figure()
    widgets.interact(draw_map,drop6=widget6,drop7=widget7, drop8=widget8)
  with tb.output_to('Correlation Matrix'):
    pylab.figure()
    widgets.interact(make_corr_plot,drop3=widget3,drop4=widget4,drop5=widget5)
  with tb.output_to('Linear Regression'):
    pylab.figure()
    widgets.interact(plot_data_linear, drop1=widget1,drop2=widget2,drop9=widget9)

create_tab('top')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

interactive(children=(Dropdown(description='X Variable:', options=('First Wave of Covid Deaths per 10k', 'Seco…

<IPython.core.display.Javascript object>

##<u>Credits</u>
**ENVS3 Team:** James Busch (Ph.D. Candidate), William Chen ('23), J.T. Erbaugh (NSF Postdoctoral Fellow), Richard Howarth (Professor of Environmental Studies)

**DIFUSE Project Managers:** Tiffany Yu ('21) and Taylor Hickey ('23)

**DIFUSE PI's:** Prof. Petra Bonfert-Taylor (Thayer School), Prof. Laura Ray (Thayer School), Prof. Scott Pauls (Mathematics), Prof. Lori Loeb (Computer Science)

**Development Team:** James Busch compiled and authored code for the web application, correlation matrix, and linear regression components. William Chen compiled and authored the code used for the interactive map.

*The DIFUSE project is supported by the National Science Foundation under grant no. DUE- 1917002*

*Please acknowledge the ENVS3 Team and the Dartmouth DIFUSE program if you share or utilize this resource*